# Task 1

---

## Web scraping and analysis

This Jupyter notebook includes some code to get you started with web scraping. We will use a package called `BeautifulSoup` to collect the data from the web. Once you've collected your data and saved it into a local `.csv` file you should start with your analysis.

### Scraping data from Skytrax

If you visit [https://www.airlinequality.com] you can see that there is a lot of data there. For this task, we are only interested in reviews related to British Airways and the Airline itself.

If you navigate to this link: [https://www.airlinequality.com/airline-reviews/british-airways] you will see this data. Now, we can use `Python` and `BeautifulSoup` to collect all the links to the reviews and then to collect the text data on each of the individual review links.

In [6]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests 

In [7]:
import requests
from bs4 import BeautifulSoup

base_url = "https://www.airlinequality.com/airline-reviews/british-airways/"
pages = 38
page_size = 100
max_entries_per_list = 3600

# Creating empty lists to collect all reviews, rating stars, date, and country
reviews = []
stars = []
date = []
country = []

# Counters to keep track of the total number of entries for each list
total_reviews = 0
total_stars = 0
total_dates = 0
total_countries = 0

for i in range(1, pages + 1):
    print(f"Scraping page {i}")

    # Create URL to collect links from paginated data
    url = f"{base_url}/page/{i}/?sortby=post_date%3ADesc&pagesize={page_size}"

    # Collect HTML data from this page
    response = requests.get(url)

    # Parse content
    content = response.content
    parsed_content = BeautifulSoup(content, 'html.parser')

    # Scraping reviews
    for para in parsed_content.find_all("div", {"class": "text_content"}):
        if total_reviews < max_entries_per_list:
            reviews.append(para.get_text())
            total_reviews += 1

    # Scraping star ratings
    for para in parsed_content.find_all("div", {"class": "rating-10"}):
        if total_stars < max_entries_per_list:
            star = para.span.get_text() if para.span else "None"
            stars.append(star)
            total_stars += 1

    # Scraping dates
    for para in parsed_content.find_all("time"):
        if total_dates < max_entries_per_list:
            date.append(para.text)
            total_dates += 1

    # Scraping countries
    for para in parsed_content.find_all("h3"):
        if total_countries < max_entries_per_list:
            country.append(para.span.next_sibling.text.strip(" ()"))
            total_countries += 1

    print(f"   ---> {total_reviews} total reviews")
    print(f"   ---> {total_stars} total stars")
    print(f"   ---> {total_dates} total dates")
    print(f"   ---> {total_countries} total countries")

    # Check if any list has reached its maximum size
    if total_reviews >= max_entries_per_list and \
       total_stars >= max_entries_per_list and \
       total_dates >= max_entries_per_list and \
       total_countries >= max_entries_per_list:
        print("Maximum number of entries reached for all lists. Stopping scraping.")
        break

# Truncate lists to ensure they contain a maximum of 3600 entries each
reviews = reviews[:max_entries_per_list]
stars = stars[:max_entries_per_list]
date = date[:max_entries_per_list]
country = country[:max_entries_per_list]

print(f"Total reviews: {len(reviews)}")
print(f"Total stars: {len(stars)}")
print(f"Total dates: {len(date)}")
print(f"Total countries: {len(country)}")


Scraping page 1
   ---> 100 total reviews
   ---> 101 total stars
   ---> 100 total dates
   ---> 100 total countries
Scraping page 2
   ---> 200 total reviews
   ---> 202 total stars
   ---> 200 total dates
   ---> 200 total countries
Scraping page 3
   ---> 300 total reviews
   ---> 303 total stars
   ---> 300 total dates
   ---> 300 total countries
Scraping page 4
   ---> 400 total reviews
   ---> 404 total stars
   ---> 400 total dates
   ---> 400 total countries
Scraping page 5
   ---> 500 total reviews
   ---> 505 total stars
   ---> 500 total dates
   ---> 500 total countries
Scraping page 6
   ---> 600 total reviews
   ---> 606 total stars
   ---> 600 total dates
   ---> 600 total countries
Scraping page 7
   ---> 700 total reviews
   ---> 707 total stars
   ---> 700 total dates
   ---> 700 total countries
Scraping page 8
   ---> 800 total reviews
   ---> 808 total stars
   ---> 800 total dates
   ---> 800 total countries
Scraping page 9
   ---> 900 total reviews
   ---> 909 to

In [11]:
df = pd.DataFrame()
df["reviews"] = reviews
df["stars"] = stars
df["date"] = date
df["country"] = country
df.describe()

,reviews,stars,date,country
count,3600,3600,3600,3600
unique,3589,12,1824,71
top,London Heathrow to Miami on one of British Air...,1,19th January 2015,United Kingdom
freq,2,867,26,2247


In [12]:
df.head()

,reviews,stars,date,country
0,✅ Trip Verified | This is the first time I ha...,\n\t\t\t\t\t\t\t\t\t\t\t\t\t5,2nd April 2024,United Kingdom
1,✅ Trip Verified | Flew business class from Do...,3,2nd April 2024,United Kingdom
2,✅ Trip Verified | Starting off at Heathrow Te...,4,28th March 2024,United Kingdom
3,Not Verified | We have flown this route with ...,8,28th March 2024,United Kingdom
4,✅ Trip Verified | A last minute business trip ...,1,26th March 2024,United Kingdom


In [13]:
df.to_csv("C:/Users/vatsa/Downloads/British-Airways-reviews-analysis/Data/BA_Data.csv")

Congratulations! Now you have your dataset for this task! The loops above collected 1000 reviews by iterating through the paginated pages on the website. However, if you want to collect more data, try increasing the number of pages!

 The next thing that you should do is clean this data to remove any unnecessary text from each of the rows. For example, "✅ Trip Verified" can be removed from each row if it exists, as it's not relevant to what we want to investigate.